In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Using OOPS concepts as asked by Sir
class AmazonScraper:
    def __init__(self, search_query="Smartphones"):
        self.search_query = search_query
        self.data = []
        self.driver = self.initialize_driver()

    def initialize_driver(self):
        options = webdriver.ChromeOptions()
        driver = webdriver.Chrome(options=options)
        return driver

    def scrape_amazon_smartphones(self):
        self.driver.get("http://amazon.in")
        time.sleep(5)

        search = self.driver.find_element(By.ID, "twotabsearchtextbox")
        search.send_keys(self.search_query)
        time.sleep(2)
        self.driver.find_element(By.ID, "nav-search-submit-button").click()

        # Loop through search pages (4 pages as asked)
        for page in range(1, 5):
            # Wait for search results to load
            WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[@data-component-type="s-search-result"]')))

            # Find product elements
            products = self.driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

            # Extract data for each product
            for product in products:
                self.extract_product_data(product)

            # Navigate to the next page
            self.go_to_next_page()

        self.driver.quit()
        return self.data

    def extract_product_data(self, product):
        # Trying try except as we have sponsored products as well as search results
        # and both were behaving differently during testing
        try:
            name = product.find_element(By.XPATH, './/h2').text
        except Exception:
            name = "N/A"

        try:
            price = product.find_element(By.XPATH, './/span[@class="a-price-whole"]').text
        except Exception:
            price = "N/A"

        try:
            rating = product.find_element(By.XPATH, './/span[@class="a-icon-alt"]').text
        except Exception:
            rating = "N/A"

        try:
            model = product.find_element(By.XPATH, './/span[@class="a-size-base"]').text
        except Exception:
            model = "N/A"

        try:
            screen_size = product.find_element(By.XPATH, './/span[@class="a-size-small"]').text
        except Exception:
            screen_size = "N/A"

        self.data.append({
            'name': name,
            'price': price,
            'rating': rating,
            'model': model,
            'screen_size': screen_size
        })

    def go_to_next_page(self):
        #Initially it was not working so again using try except to click next page
        try:
            time.sleep(2)
            self.driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]').click()
            time.sleep(2)
        except Exception:
            print("No more pages or next page button not found.")
            return

    def export_to_excel(self, filename='JorieL2Assessment.xlsx'):
        # Using dataframes as I'm using them on daily basis
        df = pd.DataFrame(self.data)
        df.to_excel(filename, index=False)

# Main execution
if __name__ == "__main__":
    scraper = AmazonScraper()
    data = scraper.scrape_amazon_smartphones()
    print(data)
    scraper.export_to_excel()
